# CryptoQuant Preprocessing


In [ ]:
!pip install hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 9.2 MB/s eta 0:00:00


In [ ]:
!pip install cybotrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 29.0 MB/s eta 0:00:00


In [ ]:
!pip install load_dotenv

In [ ]:
!pip install cybotrade-datasource

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 82.5 MB/s eta 0:00:00


In [ ]:
import cybotrade
import asyncio
import pandas as pd
import numpy as np
import os
import json
import matplotlib.pyplot as plt
from hmmlearn.hmm import GaussianHMM
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from cybotrade.strategy import Strategy
from cybotrade.models import OrderSide, Exchange, RuntimeConfig, RuntimeMode
from datetime import datetime, timezone
from cybotrade.permutation import Permutation
import cybotrade_datasource
import nest_asyncio
import os
from dotenv import load_dotenv  # Add this import at the top

In [ ]:
# Load environment variables
load_dotenv()

# Set the API key (preferably from .env file)
API_KEY =  "URPhT1mp48dLh15EjeoEZf0hTTYqpkoWQ60ZsxriEdtTJi6I"
os.environ["API_KEY"] = API_KEY

# Verify the API key
print(f"API Key set: {API_KEY[:8]}...")  # Print first 8 chars for verification

API Key set: URPhT1mp...


In [ ]:
# Define data sources configuration
data_sources = {
    'exchange_netflow': {
        'topic': 'cryptoquant|btc/exchange-flows/netflow',
        'params': {
            'window': 'hour',
            'exchange': 'all_exchange'
        }
    },
    'miner_to_exchange': {
        'topic': 'cryptoquant|btc/inter-entity-flows/miner-to-exchange',
        'params': {
            'window': 'hour',
            'from_miner': 'all_miner',
            'to_exchange': 'all_exchange'
        }
    },
    'exchange_reserve': {
        'topic': 'cryptoquant|btc/exchange-flows/reserve',
        'params': {
            'window': 'hour',
            'exchange': 'all_exchange'
        }
    },
    'price_ohlcv': {
        'topic': 'cryptoquant|btc/market-data/price-ohlcv',
        'params': {
            'window': 'hour',
            'market': 'spot',
            'exchange': 'all_exchange',
            'symbol': 'btc_usd'
        }
    }
}

async def fetch_data(api_key, topic, params, start_time, end_time):
    """Helper function to fetch data from CyboTrade"""
    full_topic = f"{topic}?{'&'.join([f'{k}={v}' for k,v in params.items()])}"
    try:
        data = await cybotrade_datasource.query_paginated(
            api_key=api_key,
            topic=full_topic,
            start_time=start_time,
            end_time=end_time
        )
        return pd.DataFrame(data) if isinstance(data, list) else None
    except Exception as e:
        print(f"Error fetching {topic}: {str(e)}")
        return None

async def main():
    try:
        if not API_KEY:
            raise ValueError("API key is not set")

        # Define time range
        start_time = datetime(year=2023, month=1, day=1, tzinfo=timezone.utc)
        end_time = datetime(year=2024, month=1, day=1, tzinfo=timezone.utc)

        # Dictionary to store all fetched dataframes
        dfs = {}

        # Fetch all data sources
        for source_name, config in data_sources.items():
            print(f"Fetching {source_name}...")
            # Skip mvrv since it doesn't support hourly data
            if source_name == 'mvrv':
                continue

            df = await fetch_data(
                API_KEY,
                config['topic'],
                config['params'],
                start_time,
                end_time
            )
            if df is not None:
                dfs[source_name] = df
                print(f"Successfully fetched {source_name} data: {len(df)} rows")
                display(df.head())
            print("-" * 100)

        return dfs  # Return the dictionary of dataframes

    except Exception as e:
        print(f"Error details: {str(e)}")
        return None

# Run the async function and store the result
nest_asyncio.apply()
dfs = await main()

Fetching exchange_netflow...
Successfully fetched exchange_netflow data: 8760 rows


,start_time,datetime,netflow_total
0,1672531200000,2023-01-01 00:00:00,309.663736
1,1672534800000,2023-01-01 01:00:00,392.516794
2,1672538400000,2023-01-01 02:00:00,469.784248
3,1672542000000,2023-01-01 03:00:00,207.986097
4,1672545600000,2023-01-01 04:00:00,-8.753805


----------------------------------------------------------------------------------------------------
Fetching miner_to_exchange...
Successfully fetched miner_to_exchange data: 8760 rows


,start_time,datetime,flow_mean,flow_total,transactions_count_flow
0,1672531200000,2023-01-01 00:00:00,0.110453,0.331358,3
1,1672534800000,2023-01-01 01:00:00,0.056179,0.730331,13
2,1672538400000,2023-01-01 02:00:00,3.873039,298.224006,77
3,1672542000000,2023-01-01 03:00:00,0.889167,20.450841,23
4,1672545600000,2023-01-01 04:00:00,0.157032,12.091448,77


----------------------------------------------------------------------------------------------------
Fetching exchange_reserve...
Successfully fetched exchange_reserve data: 8760 rows


,start_time,datetime,reserve,reserve_usd
0,1672531200000,2023-01-01 00:00:00,2.970859e+06,5.459904e+10
1,1672534800000,2023-01-01 01:00:00,2.971252e+06,5.464014e+10
2,1672538400000,2023-01-01 02:00:00,2.971722e+06,5.468251e+10
3,1672542000000,2023-01-01 03:00:00,2.971930e+06,5.460409e+10
4,1672545600000,2023-01-01 04:00:00,2.971921e+06,5.461159e+10


----------------------------------------------------------------------------------------------------
Fetching price_ohlcv...
Successfully fetched price_ohlcv data: 8760 rows


,start_time,close,datetime,high,low,open,volume
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297


----------------------------------------------------------------------------------------------------


In [ ]:
for key, df in dfs.items():
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'], utc=True)
        df.set_index('time', inplace=True)
    elif 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
        df.set_index('timestamp', inplace=True)
    dfs[key] = df.sort_index()


In [ ]:
dfs.keys()

dict_keys(['exchange_netflow', 'miner_to_exchange', 'exchange_reserve', 'price_ohlcv'])

In [ ]:
display(dfs['price_ohlcv'].head())

,start_time,close,datetime,high,low,open,volume
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297


In [ ]:
#clean data

df = dfs['price_ohlcv']
print(df.isnull().sum())

start_time    0
close         0
datetime      0
high          0
low           0
open          0
volume        0
dtype: int64


In [ ]:
df = dfs['exchange_netflow']
print(df.isnull().sum())

start_time       0
datetime         0
netflow_total    0
dtype: int64


In [ ]:
df = dfs['exchange_reserve']
print(df.isnull().sum())

start_time     0
datetime       0
reserve        0
reserve_usd    0
dtype: int64


In [ ]:
df = dfs['miner_to_exchange']
print(df.isnull().sum())

start_time                 0
datetime                   0
flow_mean                  0
flow_total                 0
transactions_count_flow    0
dtype: int64


In [ ]:
display(dfs['price_ohlcv'].head())

,start_time,close,datetime,high,low,open,volume
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297


In [ ]:
# Assuming you've already loaded the data (e.g., from CSV or API)
price_ohlcv = dfs['price_ohlcv']
exchange_netflow = dfs['exchange_netflow']
exchange_reserve = dfs['exchange_reserve']
miner_to_exchange = dfs['miner_to_exchange']

# Check columns (optional)
print(price_ohlcv.columns)
print(exchange_netflow.columns)

# Merge exchange_netflow
merged_df = price_ohlcv.merge(
    exchange_netflow[['datetime', 'netflow_total']],
    on='datetime',
    how='left'  # Keeps all price data even if netflow is missing
)

# Merge exchange_reserve
merged_df = merged_df.merge(
    exchange_reserve[['datetime', 'reserve_usd']],
    on='datetime',
    how='left'
)

# Merge miner_to_exchange
merged_df = merged_df.merge(
    miner_to_exchange[['datetime', 'flow_total']],
    on='datetime',
    how='left'
)

# Optional: Use start_time if datetime merge misses data
# merged_df = merged_df.merge(
#     miner_to_exchange[['start_time', 'flow_total']],
#     left_on='start_time',
#     right_on='start_time',
#     how='left',
#     suffixes=('', '_miner')
# )

print(merged_df.isnull().sum())  # Should show 0 NaNs if data aligns perfectly
print(merged_df.head())

Index(['start_time', 'close', 'datetime', 'high', 'low', 'open', 'volume'], dtype='object')
Index(['start_time', 'datetime', 'netflow_total'], dtype='object')
start_time       0
close            0
datetime         0
high             0
low              0
open             0
volume           0
netflow_total    0
reserve_usd      0
flow_total       0
dtype: int64
      start_time         close             datetime          high  \
0  1672531200000  16536.747967  2023-01-01 00:00:00  16564.463479   
1  1672534800000  16557.136536  2023-01-01 01:00:00  16559.355587   
2  1672538400000  16548.149805  2023-01-01 02:00:00  16570.079506   
3  1672542000000  16533.632875  2023-01-01 03:00:00  16546.046717   
4  1672545600000  16524.712159  2023-01-01 04:00:00  16575.538469   

            low          open       volume  netflow_total   reserve_usd  \
0  16503.226561  16542.783725  5516.420322     309.663736  5.459904e+10   
1  16516.182982  16522.380061  4513.341881     392.516794  5.464014e+10  

### MERGED TABLE: MERGED_DF



In [ ]:
display(merged_df.head())

,start_time,close,datetime,high,low,open,volume,netflow_total,reserve_usd,flow_total
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322,309.663736,5.459904e+10,0.331358
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881,392.516794,5.464014e+10,0.730331
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314,469.784248,5.468251e+10,298.224006
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864,207.986097,5.460409e+10,20.450841
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297,-8.753805,5.461159e+10,12.091448


In [ ]:
df = dfs['price_ohlcv']  # Get the 'price_ohlcv' DataFrame

# Calculate percentage change (returns) using price_ohlcv
returns = dfs['price_ohlcv']['close'].pct_change()

# Add 'returns' column to merged_df
merged_df['returns'] = returns

display(merged_df.head())

,start_time,close,datetime,high,low,open,volume,netflow_total,reserve_usd,flow_total,returns
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322,309.663736,5.459904e+10,0.331358,NaN
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881,392.516794,5.464014e+10,0.730331,0.001233
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314,469.784248,5.468251e+10,298.224006,-0.000543
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864,207.986097,5.460409e+10,20.450841,-0.000877
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297,-8.753805,5.461159e+10,12.091448,-0.000540


In [ ]:
# Calculate 20-day rolling volatility (annualized)
merged_df['volatility_20d'] = merged_df['returns'].rolling(20).std() * np.sqrt(252)

# Update the 'dfs' dictionary (optional) - you might want to store the merged_df
dfs['merged_df'] = merged_df  # Or update an existing key if appropriate

display(merged_df.head())

,start_time,close,datetime,high,low,open,volume,netflow_total,reserve_usd,flow_total,returns,volatility_20d
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322,309.663736,5.459904e+10,0.331358,NaN,NaN
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881,392.516794,5.464014e+10,0.730331,0.001233,NaN
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314,469.784248,5.468251e+10,298.224006,-0.000543,NaN
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864,207.986097,5.460409e+10,20.450841,-0.000877,NaN
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297,-8.753805,5.461159e+10,12.091448,-0.000540,NaN


In [ ]:
# Calculate VWAP
merged_df['vwap'] = (merged_df['volume'] * merged_df['close']).cumsum() / merged_df['volume'].cumsum()

# Update the 'dfs' dictionary (optional) - you might want to store the merged_df
dfs['merged_df'] = merged_df  # Or update an existing key if appropriate


display(merged_df.head())

,start_time,close,datetime,high,low,open,volume,netflow_total,reserve_usd,flow_total,returns,volatility_20d,vwap
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322,309.663736,5.459904e+10,0.331358,NaN,NaN,16536.747967
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881,392.516794,5.464014e+10,0.730331,0.001233,NaN,16545.922719
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314,469.784248,5.468251e+10,298.224006,-0.000543,NaN,16546.592197
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864,207.986097,5.460409e+10,20.450841,-0.000877,NaN,16543.295165
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297,-8.753805,5.461159e+10,12.091448,-0.000540,NaN,16539.334989


In [ ]:
merged_df['miner_sell_ratio'] = (merged_df['flow_total'] / merged_df['volume'])

dfs['merged_df'] = merged_df

display(merged_df.head())

,start_time,close,datetime,high,low,open,volume,netflow_total,reserve_usd,flow_total,returns,volatility_20d,vwap,miner_sell_ratio
0,1672531200000,16536.747967,2023-01-01 00:00:00,16564.463479,16503.226561,16542.783725,5516.420322,309.663736,5.459904e+10,0.331358,NaN,NaN,16536.747967,0.000060
1,1672534800000,16557.136536,2023-01-01 01:00:00,16559.355587,16516.182982,16522.380061,4513.341881,392.516794,5.464014e+10,0.730331,0.001233,NaN,16545.922719,0.000162
2,1672538400000,16548.149805,2023-01-01 02:00:00,16570.079506,16507.346758,16569.565885,4310.904314,469.784248,5.468251e+10,298.224006,-0.000543,NaN,16546.592197,0.069179
3,1672542000000,16533.632875,2023-01-01 03:00:00,16546.046717,16513.791107,16539.043868,4893.417864,207.986097,5.460409e+10,20.450841,-0.000877,NaN,16543.295165,0.004179
4,1672545600000,16524.712159,2023-01-01 04:00:00,16575.538469,16508.711671,16525.154710,5209.002297,-8.753805,5.461159e+10,12.091448,-0.000540,NaN,16539.334989,0.002321


In [ ]:
df = merged_df
df.dropna(inplace=True)
df.replace([np.inf, -np.inf], np.nan, inplace=True)

display(merged_df)

,start_time,close,datetime,high,low,open,volume,netflow_total,reserve_usd,flow_total,returns,volatility_20d,vwap,miner_sell_ratio
20,1672603200000,16607.705706,2023-01-01 20:00:00,16637.825523,16592.625651,16604.462201,5405.848739,52.396073,5.491118e+10,108.259935,-0.000134,0.014102,16556.000282,2.002645e-02
21,1672606800000,16610.269049,2023-01-01 21:00:00,16654.887421,16543.779709,16602.788182,3969.513604,92.908440,5.487459e+10,0.015708,0.000154,0.013579,16558.139499,3.957125e-06
22,1672610400000,16607.097770,2023-01-01 22:00:00,16630.574394,16566.206590,16602.722406,4326.577576,802.655095,5.488382e+10,0.164116,-0.000191,0.013394,16560.156325,3.793213e-05
23,1672614000000,16619.210720,2023-01-01 23:00:00,16636.295227,16570.394142,16598.372867,4335.352840,645.095518,5.490409e+10,0.006365,0.000729,0.012921,16562.497358,1.468141e-06
24,1672617600000,16567.236122,2023-01-02 00:00:00,16637.082598,16561.672880,16628.472985,4559.116665,-29.025021,5.484638e+10,0.002665,-0.003127,0.017496,16562.687002,5.846352e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,1704049200000,42597.709522,2023-12-31 19:00:00,42672.287951,42520.080714,42637.876496,1002.251559,10767.557906,1.286430e+11,10807.389709,-0.001243,0.048791,25389.430662,1.078311e+01
8756,1704052800000,42535.570787,2023-12-31 20:00:00,42657.708093,42489.973828,42592.930400,1070.042124,-69.981911,1.285437e+11,41.857789,-0.001459,0.046261,25389.843856,3.911789e-02
8757,1704056400000,42497.523419,2023-12-31 21:00:00,42644.775859,42492.378916,42533.137550,1026.473547,145.719970,1.285717e+11,45.135855,-0.000894,0.043245,25390.239328,4.397177e-02
8758,1704060000000,42257.026218,2023-12-31 22:00:00,42567.153583,42089.802992,42506.241293,1715.192245,-898.701783,1.278672e+11,0.432859,-0.005659,0.047629,25390.890813,2.523674e-04


In [ ]:
# prompt: create a code to implement HMM model into the merged dataset to identify implicit features to optimize trading signals based on extracted features to maximize returns.

import numpy as np
from hmmlearn.hmm import GaussianHMM

# Prepare the data for HMM
# Assuming 'merged_df' is your final DataFrame with relevant features
X = merged_df[['returns', 'volatility_20d', 'vwap', 'miner_sell_ratio']].values  # Use numerical features only

# Initialize and train the Gaussian HMM model
n_components = 3  # Number of hidden states (you might need to tune this)
hmm_model = GaussianHMM(n_components=n_components, covariance_type="diag", n_iter=100)
hmm_model.fit(X)

# Predict the hidden states (market regimes)
hidden_states = hmm_model.predict(X)

# Add the hidden states to the DataFrame
merged_df['hidden_state'] = hidden_states

# Example: Optimize trading signals based on hidden states
# (This is a basic example; you would need to tailor it to your specific strategy)
# Assuming state 0 is bearish, state 1 is neutral, and state 2 is bullish
def trading_signal(state):
  if state == 0 :
    return "Sell"
  elif state == 2:
    return "Buy"
  else:
    return "Hold"
merged_df['signal'] = merged_df['hidden_state'].apply(trading_signal)

display(merged_df.head())


#Further analysis and backtesting
# You can use the 'hidden_state' and 'signal' columns in your trading strategy
# Calculate returns based on these signals and evaluate the performance.


# Example: Backtesting and performance evaluation
# Backtesting logic (requires your trading rules)
def backtest_strategy(df):
    # Initialize portfolio value
    initial_capital = 10000
    portfolio_value = initial_capital
    position = 0

    for i in range(1, len(df)):
        if df['signal'][i] == 'Buy' and position == 0:
            position = portfolio_value / df['close'][i] # Buy the asset
            portfolio_value = 0  # All capital invested in asset
        elif df['signal'][i] == 'Sell' and position > 0:
            portfolio_value = position * df['close'][i] #Sell the asset
            position = 0  #No asset
    return portfolio_value

final_portfolio_value = backtest_strategy(merged_df)
print("Final Portfolio Value:", final_portfolio_value)


,start_time,close,datetime,high,low,open,volume,netflow_total,reserve_usd,flow_total,returns,volatility_20d,vwap,miner_sell_ratio,hidden_state,signal
20,1672603200000,16607.705706,2023-01-01 20:00:00,16637.825523,16592.625651,16604.462201,5405.848739,52.396073,5.491118e+10,108.259935,-0.000134,0.014102,16556.000282,2.002645e-02,2,Buy
21,1672606800000,16610.269049,2023-01-01 21:00:00,16654.887421,16543.779709,16602.788182,3969.513604,92.908440,5.487459e+10,0.015708,0.000154,0.013579,16558.139499,3.957125e-06,2,Buy
22,1672610400000,16607.097770,2023-01-01 22:00:00,16630.574394,16566.206590,16602.722406,4326.577576,802.655095,5.488382e+10,0.164116,-0.000191,0.013394,16560.156325,3.793213e-05,2,Buy
23,1672614000000,16619.210720,2023-01-01 23:00:00,16636.295227,16570.394142,16598.372867,4335.352840,645.095518,5.490409e+10,0.006365,0.000729,0.012921,16562.497358,1.468141e-06,2,Buy
24,1672617600000,16567.236122,2023-01-02 00:00:00,16637.082598,16561.672880,16628.472985,4559.116665,-29.025021,5.484638e+10,0.002665,-0.003127,0.017496,16562.687002,5.846352e-07,2,Buy


KeyError: 1